In [150]:
import random

In [151]:
# TODO: Smart Contract - implement CRUD operations for pool.hash and pool.id and deploy it
# ?? is update and delete even necessary?
# TODO: design the bernoulli experiment
# TODO: time and cost measurment for every write transaction
# TODO time measurement for every read transaction

In [152]:
# creation of N = 10k objects and assigning them corruption rate from 1% to 20% 
N = 10000
c_max = 0.2
c_min = 0

class Object:
    def __init__(self,pool,hash,corruption):
        # reference to the pool, integer from 1 - inf 
        self.pool=pool
        # sha256 string
        self.hash=hash
        # corruption rate from 0.01 to 0.2
        self.corruption=corruption

        self.is_corruped = False
        
objects = [Object(0,hash(i),random.uniform(c_min,c_max)) for i in range(0,N)]

In [153]:
# RQ 2
# finding the optimal pool size "k" with a bernoulli experiment based on the corruption rate
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8242460/
def optimal_size(objects):
    # TODO
    return 10
k = optimal_size(objects)

In [154]:
# creation of J ceil(N/k) pools, fill them with objects and assign each object to a pool
class Pool:
    def __init__(self,objects,id):
        # list of objects in the pool
        self.objects=objects
        # sha256 root hash of the hash-list
        self.hash=hash(objects)
        # reference to the pool, integer from 1 - inf 
        self.id=id
        
def pool_creation(N,k):
    global objects
    # TODO create pools and assign objects witht he pool id
    return []
pools = pool_creation(N,k)

In [155]:
# get contract from blockchain in order to interact with it
import json
import web3

ganache_url = 'http://127.0.0.1:7545'
w3 = web3.Web3(web3.HTTPProvider(ganache_url))

sender = w3.eth.accounts[0]
print("Sender Account: " + sender)

compiled_contract_path = '../sol/build/contracts/FixityStorage.json'
deployed_contract_address = '0x1b247D057272a6ce5c3048CcbeFC9e8037817eA8'
print("Contract Deployed at:\t" + deployed_contract_address)

with open(compiled_contract_path) as file:
    contract_json = json.load(file)  # load contract info as JSON
    contract_abi = contract_json['abi']  # fetch contract's abi - necessary to call its functions

# Fetch deployed contract reference
contract = w3.eth.contract(address=deployed_contract_address, abi=contract_abi)
contract.all_functions()

Sender Account: 0x4741BD80286561620C69d12A7Da0f6C2Ab2CEEc2
Contract Deployed at:	0x1b247D057272a6ce5c3048CcbeFC9e8037817eA8


[<Function getPoolHash(uint32)>, <Function setPoolHash(uint32,bytes32)>]

In [159]:
# metadata for every ingestion transaction
gas = 2000000
gasPrice = w3.toWei('50', 'gwei')

metaTx = {
    #"nonce":w3.eth.getTransactionCount(sender) nonce is set on transaction call
    "from":sender,
    "to":deployed_contract_address,
    "gas": gas,
    "gasPrice": gasPrice
}
metaTx

{'from': '0x4741BD80286561620C69d12A7Da0f6C2Ab2CEEc2',
 'to': '0x1b247D057272a6ce5c3048CcbeFC9e8037817eA8',
 'gas': 2000000,
 'gasPrice': 50000000000}

In [160]:
write_tx_count = 0
read_tx_count = 0

In [166]:
poolHash = "18ac3e7343f016890c510e93f935261169d9e3f565436429830faf0934f4f8e4"
poolId = 2
# set to blockchain
metaTx.update({"nonce":w3.eth.getTransactionCount(sender)})
print(metaTx)
tx_hash = contract.functions.setPoolHash(poolId,poolHash).transact(metaTx)
write_tx_count = write_tx_count + 1 
print("Transaction " + tx_hash.hex() + " succeeded")

{'from': '0x4741BD80286561620C69d12A7Da0f6C2Ab2CEEc2', 'to': '0x1b247D057272a6ce5c3048CcbeFC9e8037817eA8', 'gas': 2000000, 'gasPrice': 50000000000, 'nonce': 74}
Transaction 0x46b8af49a1eebdcaff5cf25ce6330f55d6e76f924f5b15ab14339bd5c8bf3e29 succeeded


In [167]:
# retrieve from blockchain
poolHashBytes = contract.functions.getPoolHash(poolId).call()
read_tx_count = read_tx_count + 1
print("SHA256 "+poolHashBytes.hex()+" of poolId: " + str(poolId))

SHA256 18ac3e7343f016890c510e93f935261169d9e3f565436429830faf0934f4f8e4 of poolId: 2


In [ ]:
# TODO set J pool hashes on the blockchain

In [169]:
# "ingest" objects into the "archive"
import random

class Archive:
    def __init__(self,objects):
        self.objects=objects

    def retrieveObj(self,objectHash):
        return next(obj for obj in objects if obj.hash == objectHash)

    def retrievePool(self,pool_id):
        return [obj for obj in self.objects if obj.pool == pool_id]
    
    def get_sample(self,n):
        return random.sample(self.objects,n)

    def repair(self,poolId):
        # TODO what happens if a corrupt pool was found
        return 0

archive = Archive(objects)

In [ ]:
# retrieve 100 "randomly" selected objects from the "archive" and check if check if they are corrupted 
# by comparing the newly computed pool hash with that one on the blockchain

for i in range(0,100):
    sample = archive.get_object(i)
    newly_created_pool = archive.objects_of_pool(sample.pool)
    online_pool = contract.get(sample.pool)
    
    if(online_pool.hash != newly_created_pool.hash):
        sample.is_corruped=True
        archive.repair(newly_created_pool)